## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bincore8_0_x = np.load('x_bc8_0.npy')
bincore8_1_x = np.load('x_bc8_1.npy')
bincore8_2_x = np.load('x_bc8_2.npy')
bincore8_3_x = np.load('x_bc8_3.npy')

bincore8_0_y = np.load('y_bc8_0.npy')
bincore8_1_y = np.load('y_bc8_1.npy')
bincore8_2_y = np.load('y_bc8_2.npy')
bincore8_3_y = np.load('y_bc8_3.npy')

x_bincore8 = np.concatenate((bincore8_0_x, bincore8_1_x, bincore8_2_x, bincore8_3_x), axis=0)
y_bincore8 = np.concatenate((bincore8_0_y, bincore8_1_y, bincore8_2_y, bincore8_3_y), axis=0)

print(x_bincore8.shape, y_bincore8.shape)

(328516, 96, 256) (328516, 96, 1)


In [2]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bincore8.shape[0])

x_bincore8 = x_bincore8[p]
y_bincore8 = y_bincore8[p]

print(x_bincore8.shape, y_bincore8.shape)

(328516, 96, 256) (328516, 96, 1)


## (7) 모델

In [3]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,96, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(144, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [4]:

model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

import time
print('======Training stage======')
starttime = time.time()

model1.fit(x_bincore8,
           y_bincore8,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

endtime = time.time()

print('걸린시간:', endtime - starttime)
model1.save('gcc8_bin_core_s96_h144_o0123.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 96, 288)           461952    
_________________________________________________________________
time_distributed (TimeDistri (None, 96, 1)             289       
Total params: 462,241
Trainable params: 462,241
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 229961 samples, validate on 98555 samples
Epoch 1/500
229961/229961 [==============================] - 137s 596us/sample - loss: 0.0042 - accuracy: 0.9989 - val_loss: 0.0013 - val_accuracy: 0.9997
Epoch 2/500
229961/229961 [==============================] - 138s 599us/sample - loss: 9.3839e-04 - accuracy: 0.9998 - val_loss: 8.5685e-04 - val_accura

걸린시간: 2699.5712699890137
save 완료
